In [14]:
import json

from jinja2 import Template

In [5]:
MANAGEMENT_LEVEL_TITLE_PROMPT = """
You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.

Definition of a Functional Area:
- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.

Definition of Management Level:
- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.
- Management levels include: "Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," and "Junior."

Definition of a Job Title:
- A job title refers to a specific employment position combined with a functional area.
- Examples include 'VP of Engineering' (functional area: Engineering) and 'Director of Finance' (functional area: Finance).

Instructions:
1. Management Levels: Only return management levels that match the predefined set: ["Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," "Junior"].
2. Job Titles: Normalize the job title after extracting it from the text. For example, convert "ceo" to "Chief Executive Officer" and include both the full title and its abbreviation if mentioned in the query, e.g., "VP of Engineering" and "Vice President of Engineering."
3. Response Format: Your response must be a dictionary with two keys: "management_levels" and "titles". Each key should have a list of management levels and titles respectively.
4. If a keyword is classified as title, don't include it in the management levels and vice versa. e.g if "VP of Engineering" is classified as title then don't include "VP" in management levels.

Examples:

Query: The CEO of the company made a statement
Expected Output: {"management_levels": [], "titles": ["Chief Executive Officer", "CEO"]}

Query: Provide a list of CFOs and VPs working in the technology sector
Explanation: Finance in CFO is a functional area.
Expected Output: {"management_levels": ["VP"], "titles": ["Chief Financial Officer", "CFO"]}

Query: Leaders of the top 10 revenue-based companies in the United States
Explanation: Leaders is not a title rather a hierarchical position of the highest order. These are always Csuits and Presidents. 
Expected Output: {"management_levels": ["Csuite and President"], "titles": []}

Query: Founders of the fastest-growing Middle Eastern tech startups
Explanation: Founder is a title without any functional area.
Expected Output: {"management_levels": [], "titles": ["Founder"]}

Query: Managing Directors of top automotive companies in Germany
Expected Output: {"management_levels": [], "titles": ["Managing Director"]}

Query: Hassan Waqar at QLU
Expected Output: {"management_levels": [], "titles": []}

Query: Current VPS of the largest industrial manufacturers who were once Directors at top tech companies
Expected Output: {"management_levels": ["VP", "Director"], "titles": []}

Query: Heads of sales at fortune 500 companies or who were vp of sales at apple
Expected Output: {"management_levels": [], "titles": ["Head of Sales", "VP of Sales", "Vice President of Sales"]}

Query: executives with "commercial" in title, marketplaces, based in European countries
Expected Output: {"management_levels": ["Executive and Sr. VP"], "titles": ["Commercial"]}

Note:
- Always use your large knowledge base to make educated guesses about the management levels and titles.

Now write the output for the following query:
Query: "{{query}}"
Output: 
"""

In [6]:
OPTIMIZED_PROMPT = """You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.

Definition of a Functional Area:
- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.

Definition of Management Level:
- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.
- Management levels include: "Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," and "Junior."

Definition of a Job Title:
- A job title refers to a specific employment position combined with a functional area.
- Examples include 'VP of Engineering' (functional area: Engineering) and 'Director of Finance' (functional area: Finance).

Instructions:
1. Management Levels: Only return management levels that match the predefined set: ["Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," "Junior"].
2. Job Titles: Normalize the job title after extracting it from the text. For example, convert "ceo" to "Chief Executive Officer" and include both the full title and its abbreviation if mentioned in the query, e.g., "VP of Engineering" and "Vice President of Engineering."
3. Response Format: Your response must be a dictionary with two keys: "management_levels" and "titles". Each key should have a list of management levels and titles respectively.
4. If a keyword is classified as title, don't include it in the management levels and vice versa. e.g if "VP of Engineering" is classified as title then don't include "VP" in management levels.

Example:
Query: "Provide a list of CFOs and VPs working in the technology sector"
Output: {"management_levels": ["VP"], "titles": ["Chief Financial Officer", "CFO"]}

Guidelines:
- "Leaders" implies CSuite/President level
- Titles without functional areas (e.g., "Founder", "Managing Director") go in titles list
- Use knowledge base for educated guesses about classifications

Query: "{{query}}"
Output:"""

In [7]:
with open("over_boarding_dataset.jsonl") as f:
    dataset = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'over_boarding_dataset.json'

In [29]:
with open("tmp.json") as f:
    tmp = json.load(f)

In [51]:
with open("empty_queries.json") as f:
    empty_queries = json.load(f)

In [30]:
len(tmp)

217

In [23]:
len(dataset)

426

In [52]:
len(empty_queries)

60

In [35]:
formatted_data = []

In [ ]:
{'query': 'List all Managers in the retail industry.', 'output': {'management_level': ['Manager'], 'title': []}}
{'query': 'Find all VPs in tech companies', 'output': {'management_level': ['VP'], 'title': []}}

In [54]:
len(formatted_data)

643

In [36]:
for item in dataset:
    formatted_dict = {
        "query" : item["query"],
        "output" : {
            "management_level" : item["output"][0][1],
            "title" : item["output"][1][1]    
        }
    }
    formatted_data.append(formatted_dict)

In [37]:
for item in tmp:
    formatted_data.append(item)

In [56]:
for item in empty_queries:
    formatted_dict = {
        "query" : item,
        "output" : {
            "management_level" : [],
            "title" : [] 
        }
    }
    formatted_data.append(formatted_dict)

In [57]:
len(formatted_data)

703

In [47]:
key_dict_list = []

In [48]:
for index, item in enumerate(dataset):
    if item["query"] in key_dict_list:
        print(index)
    else:
        key_dict_list.append(item["query"])


In [8]:
len(key_dict_list)

217

In [39]:
def validate_dictionary(input_dict):
    """
    Validates the format of the input dictionary.

    Args:
        input_dict: The dictionary to validate.

    Returns:
        True if the dictionary is valid, False otherwise.
    """

    # Check if the input is a dictionary
    if not isinstance(input_dict, dict):
        return False

    # Check for required keys
    if "query" not in input_dict or "output" not in input_dict:
        return False

    # Check the type of the 'query' key
    if not isinstance(input_dict["query"], str):
        return False

    # Check the structure and types within the 'output' key
    output = input_dict["output"]
    if not isinstance(output, dict):
        return False
    
    if "management_level" not in output or "title" not in output:
        return False

    if not isinstance(output["management_level"], list) or not all(isinstance(item, str) for item in output["management_level"]):
        return False
    
    if not isinstance(output["title"], list) or not all(isinstance(item, str) for item in output["title"]):
        return False

    return True


In [40]:
for item in formatted_data:
    if validate_dictionary(item) is False:
        print(item)

In [18]:
PHI_TEMPLATE = """<|system|>
{{system}}<|end|>
<|user|>
{{user}}<|end|>
<|assistant|>{{assistant}}<|end|>"""


In [3]:
SYSTEM = """You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is."""
USER = """
Definition of a Functional Area:
- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.

Definition of Management Level:
- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.
- Management levels include: "Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," and "Junior."

Definition of a Job Title:
- A job title refers to a specific employment position combined with a functional area.
- Examples include 'VP of Engineering' (functional area: Engineering) and 'Director of Finance' (functional area: Finance).

Instructions:
1. Management Levels: Only return management levels that match the predefined set: ["Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," "Junior"].
2. Job Titles: Normalize the job title after extracting it from the text. For example, convert "ceo" to "Chief Executive Officer" and include both the full title and its abbreviation if mentioned in the query, e.g., "VP of Engineering" and "Vice President of Engineering."
3. Response Format: Your response must be a dictionary with two keys: "management_levels" and "titles". Each key should have a list of management levels and titles respectively.
4. If a keyword is classified as title, don't include it in the management levels and vice versa. e.g if "VP of Engineering" is classified as title then don't include "VP" in management levels.

Example:
Query: "Provide a list of CFOs and VPs working in the technology sector"
Output: {"management_levels": ["VP"], "titles": ["Chief Financial Officer", "CFO"]}

Guidelines:
- "Leaders" implies CSuite/President level
- Titles without functional areas (e.g., "Founder", "Managing Director") go in titles list
- Use knowledge base for educated guesses about classifications

Query: "{{query}}"
Output:"""


In [8]:
import json

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return data

In [9]:
loaded_data = load_jsonl("onboarding_task_dataset.jsonl")

In [25]:
new_formatted_data = []

In [26]:
for item in loaded_data:
    user_message = Template(USER).render({"query": item["query"]})
    render_data = {
        "system": SYSTEM,
        "user": user_message,
        "assistant": item["output"]
    }
    new_formatted_data.append({"text":Template(PHI_TEMPLATE).render(render_data)})

In [27]:
print(new_formatted_data[0])

{'text': '<|system|>\nYou are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.<|end|>\n<|user|>\n\nDefinition of a Functional Area:\n- A functional area is a department or group of personnel tasked with a specific organizational function. These include departments like finance, marketing, engineering, etc.\n\nDefinition of Management Level:\n- A management level refers to a hierarchical position within an organization without a specific functional area. It encompasses broader titles that may include roles across different functional areas.\n- Management levels include: "Board of Directors," "CSuite and President," "Executive and Sr. VP," "General Manager," "VP," "Director," "Manager," "Senior (Individual Contributor)," "Mid (Individual Contributor)," and "Junior."\n\nDefinition of a Job Title:\n- A job title refers to a specific employment position combined with a functio

In [28]:
with open("onboarding_task_dataset_v3.jsonl", 'w', encoding='utf-8') as f:
        for item in new_formatted_data:
            json_line = json.dumps(item)
            f.write(json_line + '\n')

In [31]:
count = 0
for item in new_formatted_data:
    count += len(item["text"])
print("Total Training Tokens:", count)

Total Training Tokens: 1819682
